In [142]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.statespace.tools import (
    constrain_stationary_univariate, unconstrain_stationary_univariate, constrain_stationary_multivariate)

In [215]:
class PersistentDividendModel(sm.tsa.statespace.MLEModel):
    def __init__(self, endog, k_states, k_posdef):
        """
        Parameters:
        - endog: array-like, shape (n_obs, n_series)
        - k_states: number of states
        - k_posdef: dimension of the state innovation vector
        """

        # Initialize the base class (MLEModel)
        super().__init__(endog, k_states=k_states, k_posdef=k_posdef)

        self.nobs, self.n_series = self.endog.shape

        # Design matrix Z: shape (n_series, k_states)
        self['design'] = self.buildH(40, 0.967, np.full(5, .5),np.full(5, .5))

        # Transition matrix T: shape (k_states, k_states)
        self['transition'] = self.buildF(np.full(5, .5), np.full(5, .5))

        # Selection matrix R: shape (k_states, k_posdef)
        self['selection'] = np.ones((k_states, k_posdef))

        # State covariance matrix Q: shape (k_posdef, k_posdef)
        self['state_cov'] = self.buildQ(np.ones(8))

        # Observation covariance matrix H: shape (n_series, n_series)
        self['obs_cov'] = np.eye(self.n_series)

    def update(self, params, **kwargs):
        """
        Update the parameters in the state space matrices given `params`.
        """

        params = np.array(params)
        # Design matrix Z: shape (n_series, k_states)
        self['design'] = self.buildH(40, 0.967, params[0:5] , params[5:10])

        # Transition matrix T: shape (k_states, k_states)
        self['transition'] = self.buildF(params[0:5] , params[5:10])

        # Observation covariance matrix H: shape (n_series, n_series)
        self['obs_cov'] = np.diag(params[10:15])

        # State covariance matrix Q: shape (k_posdef, k_posdef)
        self['state_cov'] = self.buildQ(params[15:])

    def start_params(self):
        """
        Initial guess of parameters.
        """
        ar = np.full(10, .5)
        measurement_var = np.var(self.endog, axis=0) * 0.1
        state_var = np.array([measurement_var[3], measurement_var[2], measurement_var[4], measurement_var[3], measurement_var[2], measurement_var[4], measurement_var[0], measurement_var[1]]) * 0.9

        return np.concatenate((ar, measurement_var, state_var))

    def transform_params(self, unconstrained):
        """
        Map unconstrained parameters to constrained space.
        E.g., variances must be positive.
        """
        constrained = np.zeros(23) #unconstrained.copy()
        constrained[0:10] = constrain_stationary_univariate(unconstrained[0:10])
        constrained[10:] = unconstrained[10:] ** 2

        return constrained

    def untransform_params(self, constrained):
        """
        Map constrained parameters back to unconstrained space.
        """
        unconstrained = np.zeros(23)
        unconstrained[0:10] = unconstrain_stationary_univariate(constrained[0:10])
        unconstrained[10:] = constrained[10:] ** 0.5

        return unconstrained
        

    def param_names(self):
        """
        Names for each parameter for reference.
        """
        
        state_names = ["dp", "rp", "πp", "da", "ra", "πa", "ea", "τa"]
        observations_names = ["p", "il", "is", "d", "π"]

        # Equivalent of Julia's state_names[::4] (i.e., every 4th element starting at index 0)
        ar_names = [f"θ{lag}_{v}" for lag in range(1, 3) for v in state_names[3:]]
        measurement_sigmas = [f"σ_{n}" for n in observations_names]
        states_sigmas = [f"v{n}" for n in state_names]

        return ar_names + measurement_sigmas + states_sigmas
    
    
    def buildH(self, n, rho, theta1, theta2):
        Hp = np.array([
            1 / (1 - rho),
            -1 / (1 - rho),
            0,
            theta1[0] / (1 - rho * theta1[0]),
            -theta1[1] / (1 - rho * theta1[1]),
            0,
            -theta1[3] / (1 - rho * theta1[3]),
            0,
            theta2[0] / (1 - rho * theta2[0]),
            -theta2[1] / (1 - rho * theta2[1]),
            0,
            -theta2[3] / (1 - rho * theta2[3]),
            0
        ])

        Hin = np.array([
            0,
            1,
            1,
            0,
            (1 / n) * ((1 - theta1[1]**n) / (1 - theta1[1])) * theta1[1],
            (1 / n) * ((1 - theta1[2]**n) / (1 - theta1[2])) * theta1[2],
            0,
            (1 / n) * ((1 - theta1[4]**(n - 1)) / (1 - theta1[4])) * theta1[4],
            0,
            (1 / n) * ((1 - theta2[1]**n) / (1 - theta2[1])) * theta2[1],
            (1 / n) * ((1 - theta2[2]**n) / (1 - theta2[2])) * theta2[2],
            0,
            (1 / n) * ((1 - theta2[4]**(n - 1)) / (1 - theta2[4])) * theta2[4]
        ])

        Hi = np.array([
            0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0
        ])

        Hd = np.array([
            1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0
        ])

        Hpi = np.array([
            0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
        ])

        H = np.vstack([Hp, Hin, Hi, Hd, Hpi])
        return H
    
    def buildF(self, theta1, theta2):
        I3 = np.eye(3)
        I5 = np.eye(5)
        Z35 = np.zeros((3, 5))
        Z53 = np.zeros((5, 3))
        Z55 = np.zeros((5, 5))
        D1 = np.diag(theta1)
        D2 = np.diag(theta2)

        top = np.hstack((I3, Z35, Z35))
        middle = np.hstack((Z53, D1, D2))
        bottom = np.hstack((Z53, I5, Z55))

        return np.vstack((top, middle, bottom))
    
    def buildQ(self, Q):
        result = np.zeros((13, 13))
        cov_matrix = np.diag(Q)
        result[:8, :8] = cov_matrix
        return result


In [204]:
data_source = pd.read_excel("usa_var_data.xlsx")
data_source.set_index("Date", inplace=True)

data_quarterly = data_source[["Yeld_10Y", "T_Bill", "Dividend", "Stock_Price", "CPI"]].resample('Q').last()

data_quarterly[["Yeld_10Y", "T_Bill"]] = data_quarterly[["Yeld_10Y", "T_Bill"]] / 4

data_quarterly["CPIQ"] = data_quarterly["CPI"].pct_change()

data_quarterly["DivGrowth"] = data_quarterly["Dividend"].pct_change() - data_quarterly["CPIQ"]

data_quarterly["P/D"] = np.log(data_quarterly["Stock_Price"]/data_quarterly["Dividend"])

data_quarterly = data_quarterly.loc["1953-06-30":"1999-03-31"] 

data = data_quarterly[["P/D", "Yeld_10Y", "T_Bill", "DivGrowth", "CPIQ"]] - data_quarterly[["P/D", "Yeld_10Y", "T_Bill", "DivGrowth", "CPIQ"]].mean()



data = data * 100
display(data_quarterly[["P/D", "Yeld_10Y", "T_Bill", "DivGrowth", "CPIQ"]].mean())
display(data.tail(10))
np.round(data.describe(), decimals=4)


P/D          3.372585
Yeld_10Y     0.016942
T_Bill       0.013688
DivGrowth    0.003453
CPIQ         0.010044
dtype: float64

,P/D,Yeld_10Y,T_Bill,DivGrowth,CPIQ
Date,,,,,
1996-12-31,56.395252,-0.086721,-0.141332,0.596402,-0.499072
1997-03-31,55.164042,0.035779,-0.083832,-0.100452,-0.313053
1997-06-30,73.512889,-0.066721,-0.136332,0.285004,-0.817176
1997-09-30,75.159399,-0.164221,-0.131332,0.086524,-0.319084
1997-12-31,75.378173,-0.256721,-0.078832,0.715995,-1.004442
1998-03-31,88.467014,-0.276721,-0.111332,0.383932,-0.447511
1998-06-30,90.700592,-0.334221,-0.123832,0.917253,-0.573673
1998-09-30,78.417089,-0.584221,-0.216332,0.101015,-0.514246
1998-12-31,96.718745,-0.531721,-0.271332,0.193204,-0.821515


,P/D,Yeld_10Y,T_Bill,DivGrowth,CPIQ
count,184.0000,184.0000,184.0000,184.0000,184.0000
mean,-0.0000,0.0000,0.0000,0.0000,-0.0000
std,29.4709,0.7067,0.7087,1.2988,0.8650
min,-59.7922,-1.0917,-1.2088,-4.5835,-1.9169
25%,-20.1314,-0.6343,-0.5051,-0.8957,-0.5624
50%,1.4297,-0.0180,-0.1001,-0.0796,-0.1660
75%,14.2869,0.3339,0.3999,0.7739,0.3259
max,102.3596,2.2658,2.5037,4.2714,3.1087


In [216]:
model = PersistentDividendModel(data, 13, 13)

params = model.start_params()
params

array([5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 5.00000000e-01, 5.00000000e-01,
       5.00000000e-01, 5.00000000e-01, 8.63810795e+01, 4.96665599e-02,
       4.99560999e-02, 1.67766098e-01, 7.44089889e-02, 1.50989488e-01,
       4.49604899e-02, 6.69680900e-02, 1.50989488e-01, 4.49604899e-02,
       6.69680900e-02, 7.77429716e+01, 4.46999039e-02])

In [218]:
res = model.fit()

res.summary()

TypeError: bad operand type for unary -: 'method'

In [140]:
# Θ1 and Θ2 as NumPy arrays
theta1 = np.array([0.333, 0.343, 0.003, 0.528, 0.234])
theta2 = np.array([0.338, 0.286, -0.130, -0.012, 0.625])

# R as a NumPy array
R = np.array([1.45e-1, 1.59e-14, 3.01e-13, 7.43e-10, 3.78e-12])

# Q as a NumPy array, scaled by 1/100
Q = np.array([4.66e-2, 2.34e-2, 5.51e-2, 2.2, 0.5, 0.253, 62.11, 2.09]) / 100

test_params = np.r_[theta1, theta2, R, Q]
np.round(test_params, decimals=3)

test_model = PersistentDividendModel(np.full((250, 5), .1), 13, 13)

test_model.simulate(test_params, 1)

RuntimeError: Statespace model not initialized.